## Scrapping bantennews.co.id

In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [19]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []
keywords='anies'

In [20]:
def scrape_links(page_number,keywords):
    url=f"https://www.bantennews.co.id/page/{page_number}/?s={keywords}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('h3',{"class": "entry-title td-module-title"})
    
    page_links = []
    for article in articles:
        link_element = article.find('a')
        if link_element is not None:
            link = link_element.get('href')
            if link is not None:
                page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [21]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 10 links from page 1
Total Links: 10


In [22]:
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "entry-title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            # author_elem = soup.find('div', {"class": "tags-wrapper"})
            # if author_elem:
            #     author_text = author_elem.find('span')
            #     author_text = author_text.text.split(':')[1]
            # else:
            author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('time', {"class": "entry-date updated td-module-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('li', {"class": "entry-category"})
            if category_elements:
                category_text = category_elements.find('a')
                category_text= category_text.text.strip()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "td-post-content td-pb-padding-side"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"
            nama_berita_match = re.search(r'https://www\.(\w+)\.co\.id/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"
            results.append({'title': title_text,
                            'keywords': keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'region':'banten',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [23]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [24]:
df = pd.DataFrame(results)
print("Total Hasil ",len(results))
df.head(10)

Total Hasil  10


,title,keywords,author,category,date,content,nama_berita,region,link
0,Kandidat Kuat Cawapres Prabowo Usai Cak Imin M...,anies,Author not found,Pemilu 2024,"Senin 4 Sep 2023, 11:09 WIB",SERANG – ursa bakal calon wakil presiden (Cawa...,bantennews,banten,https://www.bantennews.co.id/kandidat-kuat-caw...
1,Politikus NasDem Sebut Masih Ada Kemungkinan D...,anies,Author not found,Pemilu 2024,"Rabu 30 Agu 2023, 09:13 WIB",JAKARTA – Anggota Fraksi Partai NasDem di DPR ...,bantennews,banten,https://www.bantennews.co.id/politikus-nasdem-...
2,"Survei Elektabilitas Capres LSI, Ganjar dan Pr...",anies,Author not found,Pemilu 2024,"Kamis 31 Agu 2023, 09:05 WIB",SERANG – Elektabilitas Ganjar Pranowo unggul t...,bantennews,banten,https://www.bantennews.co.id/survei-elektabili...
3,Eks Penasihat KPK Bicara Peluang Cak Imin Jadi...,anies,Author not found,Pemilu 2024,"Rabu 6 Sep 2023, 06:07 WIB",SERANG – Mantan penasihat Komisi Pemberantasan...,bantennews,banten,https://www.bantennews.co.id/eks-penasihat-kpk...
4,Survei SMRC Terbaru Anies-Muhaimin Ganjar-RK d...,anies,Author not found,Pemilu 2024,"Jumat 15 Sep 2023, 04:11 WIB",SERANG – Lembaga survei Saiful Munjani Researc...,bantennews,banten,https://www.bantennews.co.id/survei-smrc-terba...
5,"Anies Baswedan Berpaling ke Cak Imin, Kader De...",anies,Author not found,Pemilu 2024,"Sabtu 2 Sep 2023, 03:15 WIB",LEBAK – Kabar mengejutkan datang dari Koalisi ...,bantennews,banten,https://www.bantennews.co.id/anies-baswedan-be...
6,"AHY Kena Tikung, Cak Imin Bakal Jadi Cawapres ...",anies,Author not found,Pemilu 2024,"Jumat 1 Sep 2023, 09:08 WIB","SERANG – Partai Demokrat dan PKS, terutama Ket...",bantennews,banten,https://www.bantennews.co.id/ahy-kena-tikung-c...
7,"Sudah Tahu Nama Cawapres di Kantong Anies, AHY...",anies,Author not found,Pemilu 2024,"Senin 28 Agu 2023, 10:13 WIB",JAKARTA – Ketua Umum Partai Demokrat Agus Hari...,bantennews,banten,https://www.bantennews.co.id/sudah-tahu-nama-c...
8,Anies Senang PKB Bergabung dengan Koalisi Peru...,anies,Author not found,Pemilu 2024,"Minggu 3 Sep 2023, 00:10 WIB","SERANG – Anies Baswedan, Bakal calon presiden ...",bantennews,banten,https://www.bantennews.co.id/anies-senang-pkb-...
9,Anies: Surya Paloh Tak Menolak AHY Jadi Bacawa...,anies,Author not found,Pemilu 2024,"Selasa 5 Sep 2023, 15:04 WIB",JAKARTA – Bakal capres Koalisi Perubahan untuk...,bantennews,banten,https://www.bantennews.co.id/anies-surya-paloh...


In [25]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 10


,title,keywords,author,category,date,content,nama_berita,region,link
0,Kandidat Kuat Cawapres Prabowo Usai Cak Imin M...,anies,Author not found,Pemilu 2024,"Senin 4 Sep 2023, 11:09 WIB",SERANG – ursa bakal calon wakil presiden (Cawa...,bantennews,banten,https://www.bantennews.co.id/kandidat-kuat-caw...
1,Politikus NasDem Sebut Masih Ada Kemungkinan D...,anies,Author not found,Pemilu 2024,"Rabu 30 Agu 2023, 09:13 WIB",JAKARTA – Anggota Fraksi Partai NasDem di DPR ...,bantennews,banten,https://www.bantennews.co.id/politikus-nasdem-...
2,"Survei Elektabilitas Capres LSI, Ganjar dan Pr...",anies,Author not found,Pemilu 2024,"Kamis 31 Agu 2023, 09:05 WIB",SERANG – Elektabilitas Ganjar Pranowo unggul t...,bantennews,banten,https://www.bantennews.co.id/survei-elektabili...
3,Eks Penasihat KPK Bicara Peluang Cak Imin Jadi...,anies,Author not found,Pemilu 2024,"Rabu 6 Sep 2023, 06:07 WIB",SERANG – Mantan penasihat Komisi Pemberantasan...,bantennews,banten,https://www.bantennews.co.id/eks-penasihat-kpk...
4,Survei SMRC Terbaru Anies-Muhaimin Ganjar-RK d...,anies,Author not found,Pemilu 2024,"Jumat 15 Sep 2023, 04:11 WIB",SERANG – Lembaga survei Saiful Munjani Researc...,bantennews,banten,https://www.bantennews.co.id/survei-smrc-terba...
5,"Anies Baswedan Berpaling ke Cak Imin, Kader De...",anies,Author not found,Pemilu 2024,"Sabtu 2 Sep 2023, 03:15 WIB",LEBAK – Kabar mengejutkan datang dari Koalisi ...,bantennews,banten,https://www.bantennews.co.id/anies-baswedan-be...
6,"AHY Kena Tikung, Cak Imin Bakal Jadi Cawapres ...",anies,Author not found,Pemilu 2024,"Jumat 1 Sep 2023, 09:08 WIB","SERANG – Partai Demokrat dan PKS, terutama Ket...",bantennews,banten,https://www.bantennews.co.id/ahy-kena-tikung-c...
7,"Sudah Tahu Nama Cawapres di Kantong Anies, AHY...",anies,Author not found,Pemilu 2024,"Senin 28 Agu 2023, 10:13 WIB",JAKARTA – Ketua Umum Partai Demokrat Agus Hari...,bantennews,banten,https://www.bantennews.co.id/sudah-tahu-nama-c...
8,Anies Senang PKB Bergabung dengan Koalisi Peru...,anies,Author not found,Pemilu 2024,"Minggu 3 Sep 2023, 00:10 WIB","SERANG – Anies Baswedan, Bakal calon presiden ...",bantennews,banten,https://www.bantennews.co.id/anies-senang-pkb-...
9,Anies: Surya Paloh Tak Menolak AHY Jadi Bacawa...,anies,Author not found,Pemilu 2024,"Selasa 5 Sep 2023, 15:04 WIB",JAKARTA – Bakal capres Koalisi Perubahan untuk...,bantennews,banten,https://www.bantennews.co.id/anies-surya-paloh...


In [26]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../../tempat_hasil_daerah/bantennews_{keywords}_{current_datetime}.xlsx'
# filtered_df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')